# Lection 3

Ниже представлен код программы в котором производится поиск книг по заданным критериям.

In [ ]:
    def search_books(books_settings):
        # Connecting to database
        conn = psycopg2.connect(
            database="Technical_Literature",
            user="postgres",
            host='127.0.0.1',
            password="25072004",
            port=5432
        )
        cur = conn.cursor()

        # quiry in case we need to make requst by all parametrs 
        query_not_all = f"""
                SELECT books.label, books.title, books.year, aditional_information.pages
                FROM books
                LEFT JOIN aditional_information ON books.book_id = aditional_information.book_id
                WHERE books.author = '{books_settings["Author"]}'
                      AND books.label = '{books_settings["Genre"]}' 
                      AND books.year > {books_settings["Year_from"]} 
                      AND books.year < {books_settings["Year_to"]};
                """
        # quiry in case we need to make request by only genre
        query_author_all = f"""
                SELECT books.label, books.title, books.year, aditional_information.pages
                FROM books
                LEFT JOIN aditional_information ON books.book_id = aditional_information.book_id
                WHERE books.label = '{books_settings["Genre"]}' 
                      AND books.year > {books_settings["Year_from"]} 
                      AND books.year < {books_settings["Year_to"]};
        """
        # quiry in case we need to make request by only author
        query_genre_all = f"""
                SELECT books.label, books.title, books.year, aditional_information.pages
                FROM books
                LEFT JOIN aditional_information ON books.book_id = aditional_information.book_id
                WHERE books.author = '{books_settings["Author"]}' 
                      AND books.year > {books_settings["Year_from"]} 
                      AND books.year < {books_settings["Year_to"]};
        """
        # quiry in case we don't have any parametrs 
        query_all = """
        SELECT books.label, books.title, books.year, aditional_information.pages
        FROM books
        LEFT JOIN aditional_information ON books.book_id = aditional_information.book_id;
        """

        # checking for request parametrs 
        if books_settings["Author"] == "ALL" and books_settings["Genre"] == "ALL":
            cur.execute(query_all)
        elif books_settings["Author"] == "ALL":
            cur.execute(query_author_all)
        elif books_settings["Genre"] == "ALL":
            cur.execute(query_genre_all)
        else:
            cur.execute(query_not_all)

        # getting data
        books = cur.fetchall()

        # closing our database connection
        cur.close()
        conn.close()

        # returning books
        return books

Для начала рассмотрим как производится запрос в базу данных.

In [ ]:
query_not_all = f"""
                SELECT books.label, books.title, books.year, aditional_information.pages
                FROM books
                LEFT JOIN aditional_information ON books.book_id = aditional_information.book_id
                WHERE books.author = '{books_settings["Author"]}'
                      AND books.label = '{books_settings["Genre"]}' 
                      AND books.year > {books_settings["Year_from"]} 
                      AND books.year < {books_settings["Year_to"]};
                """

В данном виде представлен запрос со всеми параметрами поиска в базу данных. С помощью *LEFT JOIN* мы объединяем *aditional_inforamtion* с *books*.
Далее с помощью *WHERE* мы производим поиск всех элементов совпадающих с нашими параметрами. 

Из запроса *WHERE* в зависимости от того какие параметры ввел пользователь убираются не введенные параметры. 

In [ ]:
if books_settings["Author"] == "ALL" and books_settings["Genre"] == "ALL":
    cur.execute(query_all)
elif books_settings["Author"] == "ALL":
    cur.execute(query_author_all)
elif books_settings["Genre"] == "ALL":
    cur.execute(query_genre_all)
else:
    cur.execute(query_not_all)

В данном коде происходит проверка на введенные параметры, в зависимости от этого мы делаем запрос либо по одному из параметров, либо по всем, либо без параметров.

После завершения функция возвращает результат поиска

In [ ]:
def handle_search_results(self, books):
    quiry = self.ids.search_quiry.text.lower()
    pattern = r".*" + quiry + ".*"

    self.current_books = [[genre, title, year, pages] for genre, title, year, pages in books if re.match(pattern, title.lower())]
    self.show_books()

После того как мы выполнили наш запрос данные с него попадают в данную функцию в которой с помощью регулярного выражения *re.match(pattern, title.lower())* отбираются только те данные которые соответствуют нашему запросу и записываются в переменную *self.current_books* для дальнейшего вывода на экран.

В конце данная функция вызывает другую функцию *self.show_books*, которая осуществляет вывод данных на экран.

In [ ]:
def show_books(self):
    if self.show:
        titles = [f"[{genre}]| {title}  {year}y.  {pages}p." for genre, title, year, pages in self.show]
    else:
        titles = [f"[{genre}]| {title}  {year}y.  {pages}p." for genre, title, year, pages in self.current_books]
    self.ids.scroll.text = "\n\n".join(titles[:275]) if titles != [] else "Not Found"

Данная функция выводит информацию о книгах на экран путем обновления значения text виджета Label.

In [ ]:
    def sort_by(self):
        mode = self.ids.sort_by.text

        if "None" in mode:
            self.ids.sort_by.text = "Sort by: Year"
            self.show = sorted(self.current_books, key=lambda year: year[1])
        elif "Year" in mode:
            self.ids.sort_by.text = "Sort by: Title"
            self.show = sorted(self.current_books, key=lambda title: title[0])
        elif "Title" in mode:
            self.ids.sort_by.text = "Sort by: None"
            self.show = self.current_books
        else:
            print("Can't sort!")

        self.show_books()

Данная функция производит сортировку книг либо по году выпуска, либо по названию.